In [1]:
'''
builds the diff files to be used for a big ol' daycent run on aws batch
uses the templates directory to build new graz.100, fix.100, and rainmore_exp.sch files
and insert the already generated LHS values into those files

they're numbered by "run", wherein if we're doing 10k runs it'll be diff_0 through diff_10000

the folder structure just uses the second level epoch for the containing folder in case we needed to run one of the same size

it makes a directory for each run, compresses it into a tgz, then deletes the original folder, so you'll end up with diff_0.tgz through diff_10000.tgz
'''
import matrices.LHS_1000
import os
import subprocess
import time

In [2]:
matrix = matrices.LHS_1000.matrix
# matrix

In [3]:
size = len(matrix)
# experiment or crop name or other identifier
exp = "soybeans"

In [4]:
# create schedule file with new inputs/dates based on the template
def create_sched(cult_nt_date, cult_b_date, crop_sybn2_date, crop_plant_date, fert_n4_date, harv_g_date, epoch, i):
    # read template
    with open('templates/{}.sch'.format(exp)) as f:
        lines = f.readlines()
    
    sout = open("diffs/{}/diff_{}/{}.sch".format(epoch, i, exp), "w")
    for line in lines:
        if line.startswith("1  139  CULT  NT"):
            sout.write("1  {}  CULT NTX #PSI\n".format(cult_nt_date))
        elif line.startswith("1  140  CULT  B  #planter"):
            sout.write("1  {} CULT B # PSI\n".format(cult_b_date))
        elif line.startswith("1  140  CROP  SYBN2"):
            sout.write("1  {} CROP SYBN2 # PSI\n".format(crop_sybn2_date))
        elif line.startswith("1  140  PLTM"):
            sout.write("1  {} PLTM #PSI\n".format(crop_plant_date))
        elif line.startswith("1   145   FERT   N4   # 25/05"):
            sout.write("1  {} FERT  XN4   # PSI\n".format(fert_n4_date))
        elif line.startswith("1  282  HARV  G"):
            sout.write("1  {} HARV  X # PSI\n".format(harv_g_date))
        elif line.startswith("1  281 LAST # PSI\n"):
            sout.write("1  {} LAST # PSI\n".format(harv_g_date - 1))
        else:
            sout.write(line)
    sout.close()

In [5]:
# appends new NTX onto the end of the cult.100
def create_cult(clteff_1, clteff_2, clteff_3, clteff_4, epoch, i):
    # read template
    with open('templates/cult.100') as f:
        lines = f.read()
    
    # write to the new diff file with the block appended to the end
    foutfile = open("diffs/{}/diff_{}/cult.100".format(epoch, i), "w")
    foutfile.write(lines)
    foutfile.write("NTX\n")
    foutfile.write("0 CULTRA(1)\n0 CULTRA(2)\n0 CULTRA(3)\n0 CULTRA(4)\n0 CULTRA(5)\n0 CULTRA(6)\n0 CULTRA(7)\n")
    foutfile.write("{:0.3f} CLTEFF(1)\n".format(clteff_1))
    foutfile.write("{:0.3f} CLTEFF(2)\n".format(clteff_2))
    foutfile.write("{:0.3f} CLTEFF(3)\n".format(clteff_3))
    foutfile.write("{:0.3f} CLTEFF(4)\n".format(clteff_4))
    foutfile.write("\n")
    foutfile.close()

In [6]:
# fert100 file
# creating new XN4 Fert stanza
def create_fert(feramt_n4_1, feramt_n4_2, feramt_n4_3, epoch, i):
    # read template
    with open('templates/fert.100') as f:
        lines = f.read()
    
    n4 = '''XN4 
{:0.3f} FERAMT(1)
{:0.3f} FERAMT(2)
{:0.3f} FERAMT(3)
0 AUFERT
1 NINHIB
7 NINHTM
0.5 FRAC_NH4
0.5 FRAC_NO3
'''.format(feramt_n4_1, feramt_n4_2, feramt_n4_3)
    
    
    # write to the new diff file with the block appended to the end
    foutfile = open("diffs/{}/diff_{}/fert.100".format(epoch, i), "w")
    foutfile.write(lines)
    foutfile.write(n4)
    foutfile.write("\n")
    foutfile.close()

In [7]:
# harv.100 file
def create_harv(rmvstr, remwsd, epoch, i):
    # read template
    with open('templates/harv.100') as f:
        lines = f.read()
    x = '''X 
0 AGLREM
0 BGLREM
1 FLGHRV
{:0.3f} RMVSTR
{:0.3f} REMWSD
0 HIBG'''.format(rmvstr, remwsd)
    
    # write to the new diff file with the block appended to the end
    foutfile = open("diffs/{}/diff_{}/harv.100".format(epoch, i), "w")
    foutfile.write(lines)
    foutfile.write(x)
    foutfile.write("\n")
    foutfile.close()
    

In [8]:
# set up directory structure
num_diffs = len(matrix)

dir = "./diffs"
if not os.path.isdir(dir):
    print("creating {}".format(dir))
    os.mkdir(dir)

# create new diffs/epoch folder
epoch = int(time.time())
folder_name = 'diffs/{}'.format(epoch)
print("folder_name is {}".format(folder_name))
os.mkdir(folder_name)

# create folder for each row in the matrix
for i in range(num_diffs):
    path = os.path.join(folder_name, "diff_{}".format(i))
    # print("Making dir {}".format(path))
    os.mkdir(path)

print("Made directories.")

folder_name is diffs/1665667448
Made directories.


In [9]:
%%time

# write the diff files to the diffs directory and tgz them up

i = 0
print("epoch is {}".format(epoch))
for row in matrix:
    clteff_1 = row[0]
    clteff_2 = row[1]
    clteff_3 = row[2]
    clteff_4 = row[3]
    cult_nt_date = row[4]
    cult_b_date = row[5]
    crop_sybn2_date = row[6]
    crop_plant_date = row[7]
    feramt_n4_1 = row[8]
    feramt_n4_2 = row[9]
    feramt_n4_3 = row[10]
    feramt_n4_date = row[11]
    rmvstr = row[12]
    remwsd = row[13]
    harv_g_date = row[14]
    
    # write the diff files to the numbered directories
    # create_graz100(flgrem, epoch, i)
    # create_fix(decomp, minlch, varat12, epoch, i)
    create_sched(cult_nt_date, cult_b_date, crop_sybn2_date, crop_plant_date, feramt_n4_date, harv_g_date, epoch, i)
    create_cult(clteff_1, clteff_2, clteff_3, clteff_4, epoch, i)
    create_fert(feramt_n4_1, feramt_n4_2, feramt_n4_3, epoch, i)
    create_harv(rmvstr, remwsd, epoch, i)
    
    # change to the diffs/epoch directory so we don't have a nested tgz with the parent directories
    os.chdir('diffs/{}'.format(epoch))
    
    # create a tgz of each of those directories
    dirname = 'diff_{}'.format(i)
    tarname = 'diff_{}.tgz'.format(i)
    subprocess.call(['tar', '-czf', tarname, dirname])
    subprocess.call(['rm', '-rf', dirname])
    
    os.chdir('../..')
    
    i = i + 1

print("Done")

epoch is 1665667448
Done
CPU times: user 1.01 s, sys: 3.38 s, total: 4.39 s
Wall time: 15.1 s


In [10]:
os.getcwd()

'/home/jovyan/accelerator/newdata/soy'

In [11]:
# where we at?
os.chdir(f'diffs/{epoch}')
os.getcwd()

'/home/jovyan/accelerator/newdata/soy/diffs/1665667448'

In [12]:
s3_diffs = 's3://daycent-jobs/jobs/{}_{}/diffs/'.format(exp, len(matrix))
print(s3_diffs)

s3://daycent-jobs/jobs/soybeans_1000/diffs/


In [13]:
%%time
# copy these to the right place
subprocess.call(['/home/jovyan/aws-cli/v2/current/bin/aws', 's3', 'cp', '.', s3_diffs, 
                 '--recursive', '--exclude', '"*"', '--include', '"*.tgz"'])

CPU times: user 3.86 ms, sys: 3.97 ms, total: 7.83 ms
Wall time: 16.8 s


0